In [2]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import *
import numpy as np

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#Only look at 2s and 7s
num_classes = 2
train_picks = np.logical_or(y_train==2,y_train==7)
test_picks = np.logical_or(y_test==2,y_test==7)

x_train = x_train[train_picks]
x_test = x_test[test_picks]
y_train = np.array(y_train[train_picks]==7,dtype=int)
y_test = np.array(y_test[test_picks]==7,dtype=int)


if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (12223, 28, 28, 1)
12223 train samples
2060 test samples


In [8]:
#Function that Allows the GridSearch to change the model
def create_model(activator='relu', dropout_rate1=.25, dropout_rate2=.25, 
                 _optimizer='Adam', batch_size=128, epochs=12, neurons=16, learning_rate=.001):
    
    #This allows you to tune Optimizer & Learning Rate at the Same Time
    if _optimizer == 'Adam':
        optimizer = Adam(lr=learning_rate)
    elif _optimizer == 'SGD':
        optimizer = SGD(lr=learning_rate)
    elif _optimizer == 'Adamax':
        optimizer = Adamax(lr=learning_rate)
    else:
        optimizer = Nadam(lr=learning_rate)
        
    model = Sequential()
    model.add(Conv2D(4, kernel_size=(3, 3),activation=activator,input_shape=input_shape))
    model.add(Conv2D(8, (3, 3), activation=activator))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate1))
    model.add(Flatten())
    model.add(Dense(neurons, activation=activator))
    model.add(Dropout(dropout_rate2))
    model.add(Dense(2, activation='softmax'))

    
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

In [37]:
# Lists of possible Values for Hyperparameters
batch_size = [8, 16, 32, 64, 128]
_optimizer = ['Adam', 'SGD', 'Adamax', 'Nadam']
learning_rate = [.001, .002, .005, .01]
epochs = [4, 8, 12]
activator = ['softmax', 'relu', 'tanh', 'sigmoid', 'linear']
dropout_rate1 = [.0, .1, .25]
dropout_rate2 = [.0, .1, .25, .5]
neurons = [8, 16, 32]
param_grid = dict(activator=activator, dropout_rate1=dropout_rate1, dropout_rate2=dropout_rate2,
                  _optimizer=_optimizer, batch_size=batch_size, epochs=epochs, neurons=neurons, 
                  learning_rate=learning_rate)
search_model = KerasClassifier(build_fn=create_model, verbose=1)

# I chose to tune all of the parameters together, because there is interdependency.
# Because of this, I also chose to do a random search to reduce the expense.
grid = RandomizedSearchCV(estimator=search_model, param_distributions=param_grid, random_state=3, n_jobs=1, n_iter=20)
grid_result = grid.fit(x_train, y_train)

Epoch 1/8
8148/8148 [==============================] - 4s - loss: 0.1035 - acc: 0.9703     
Epoch 2/8
8148/8148 [==============================] - 3s - loss: 0.0542 - acc: 0.9826     
Epoch 3/8
8148/8148 [==============================] - 3s - loss: 0.0379 - acc: 0.9891     
Epoch 4/8
8148/8148 [==============================] - 3s - loss: 0.0369 - acc: 0.9876     
Epoch 5/8
8148/8148 [==============================] - 4s - loss: 0.0355 - acc: 0.9883     
Epoch 6/8
8148/8148 [==============================] - 4s - loss: 0.0332 - acc: 0.9912     
Epoch 7/8
8148/8148 [==============================] - 4s - loss: 0.0320 - acc: 0.9901     
Epoch 8/8
8149/8149 [==============================] - 5s - loss: 0.0827 - acc: 0.9731     
Epoch 2/8
8149/8149 [==============================] - 4s - loss: 0.0439 - acc: 0.9853     
Epoch 3/8
8149/8149 [==============================] - 4s - loss: 0.0330 - acc: 0.9893     
Epoch 4/8
8149/8149 [==============================] - 4s - loss: 0.0315 - acc: 

8148/8148 [==============================] - 16s - loss: 0.6930 - acc: 0.5085    
Epoch 9/12
8148/8148 [==============================] - 17s - loss: 0.6929 - acc: 0.5085    
Epoch 10/12
8148/8148 [==============================] - 16s - loss: 0.6929 - acc: 0.5085    - ETA: 0s - loss: 0.6930 - acc
Epoch 11/12
8148/8148 [==============================] - 16s - loss: 0.6929 - acc: 0.5085    
Epoch 12/12
8148/8148 [==============================] - 4s     
Epoch 1/12
8149/8149 [==============================] - 18s - loss: 0.6930 - acc: 0.5163    
Epoch 2/12
8149/8149 [==============================] - 17s - loss: 0.6928 - acc: 0.5177    
Epoch 3/12
8149/8149 [==============================] - 17s - loss: 0.6927 - acc: 0.5177    
Epoch 4/12
8149/8149 [==============================] - 16s - loss: 0.6927 - acc: 0.5177    
Epoch 5/12
8149/8149 [==============================] - 16s - loss: 0.6926 - acc: 0.5177    
Epoch 6/12
8149/8149 [==============================] - 16s - loss: 0.6926 - 

8148/8148 [==============================] - 5s - loss: 0.0775 - acc: 0.9764     
Epoch 3/8
8148/8148 [==============================] - 5s - loss: 0.0649 - acc: 0.9772     
Epoch 4/8
8148/8148 [==============================] - ETA: 0s - loss: 0.0612 - acc: 0.979 - 5s - loss: 0.0614 - acc: 0.9799     
Epoch 5/8
8148/8148 [==============================] - 5s - loss: 0.0570 - acc: 0.9812     
Epoch 6/8
8148/8148 [==============================] - 5s - loss: 0.0542 - acc: 0.9816     
Epoch 7/8
8148/8148 [==============================] - 5s - loss: 0.0493 - acc: 0.9832     
Epoch 8/8
8149/8149 [==============================] - 7s - loss: 0.2244 - acc: 0.9232     
Epoch 2/8
8149/8149 [==============================] - 5s - loss: 0.0824 - acc: 0.9708     
Epoch 3/8
8149/8149 [==============================] - 5s - loss: 0.0672 - acc: 0.9748     
Epoch 4/8
8149/8149 [==============================] - 5s - loss: 0.0573 - acc: 0.9788     
Epoch 5/8
8149/8149 [==============================]

8149/8149 [==============================] - 17s - loss: 0.6946 - acc: 0.4948    
Epoch 2/4
8149/8149 [==============================] - 13s - loss: 0.6943 - acc: 0.4960    
Epoch 3/4
8149/8149 [==============================] - 13s - loss: 0.6947 - acc: 0.4901    
Epoch 4/4
12223/12223 [==============================] - 8s - loss: 0.0933 - acc: 0.9689     
Epoch 2/12
12223/12223 [==============================] - 6s - loss: 0.0447 - acc: 0.9869     
Epoch 3/12
12223/12223 [==============================] - 6s - loss: 0.0341 - acc: 0.9895     
Epoch 4/12
12223/12223 [==============================] - 7s - loss: 0.0287 - acc: 0.9911     
Epoch 5/12
12223/12223 [==============================] - 8s - loss: 0.0246 - acc: 0.9925     
Epoch 6/12
12223/12223 [==============================] - 7s - loss: 0.0204 - acc: 0.9939     
Epoch 7/12
12223/12223 [==============================] - 6s - loss: 0.0190 - acc: 0.9951     
Epoch 8/12
12223/12223 [==============================] - 6s - loss: 0

In [39]:
# What was the very best the result?
grid_result.best_score_

0.99443671752688123

In [40]:
# Ensure you don't have overfitting by checking against the test set
grid_result.score(x_test, y_test)

1984/2060 [===========================>..] - ETA: 0s

0.99126213592233015

In [41]:
# Display the final parameters
grid_result.best_params_

{'_optimizer': 'Adamax',
 'activator': 'relu',
 'batch_size': 64,
 'dropout_rate1': 0.25,
 'dropout_rate2': 0.1,
 'epochs': 12,
 'learning_rate': 0.005,
 'neurons': 8}

In [45]:
# To let you know that the 1.5 hour job is complete
from IPython.display import Audio
Audio(filename='./JobDone.mp3', autoplay=True)

In [42]:
# Just in case you want to see the parameters/combinations checked
grid_result.cv_results_

{'mean_fit_time': array([  35.42122801,  180.54069916,  208.0674359 ,   28.82937503,
          32.49717347,   37.71703704,  216.42127911,   19.9575278 ,
         113.43821383,   59.78622007,   38.95401669,   62.60841735,
         123.44176221,   47.89012225,   19.32160266,  162.77963122,
          24.54693659,   58.32220109,   54.70594843,   60.1191566 ]),
 'mean_score_time': array([ 1.55711532,  2.40351176,  3.04836106,  1.52377597,  2.78812178,
         1.78898096,  3.88805199,  1.867546  ,  3.28094538,  2.69291186,
         2.07740641,  2.76209879,  3.94490997,  2.70156272,  2.10820707,
         2.78730138,  2.8044219 ,  2.35423303,  2.44961476,  3.43155909]),
 'mean_test_score': array([ 0.99190052,  0.98985519,  0.51255829,  0.99329134,  0.98625542,
         0.99190052,  0.51255829,  0.51255829,  0.99050969,  0.95246666,
         0.98429191,  0.98641905,  0.99337315,  0.9836374 ,  0.99361859,
         0.98699174,  0.989937  ,  0.99443672,  0.98830074,  0.50274074]),
 'mean_train_sc